In [100]:
import pandas as pd
import numpy as np
import nltk
import re

from nltk.sentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

In [78]:
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Leandro
[nltk_data]     Starke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Leandro
[nltk_data]     Starke\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Leandro
[nltk_data]     Starke\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [79]:
df = pd.read_csv('imdb.csv', sep=';')
df.head()

,"review,sentiment",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,One of the other reviewers has mentioned that ...,NaN,NaN,NaN,NaN,NaN,NaN
1,A wonderful little production. <br /><br />The...,NaN,NaN,NaN,NaN,NaN,NaN
2,I thought this was a wonderful way to spend ti...,NaN,NaN,NaN,NaN,NaN,NaN
3,Basically there's a family where a little boy ...,NaN,NaN,NaN,NaN,NaN,NaN
4,"Petter Mattei's ""Love in the Time of Money"" is...",NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
def get_sentiment(x: str) -> int:

  comma_pos = x.rfind(',') + 1

  if x[comma_pos::] == 'positive':
    return 1
  elif x[comma_pos::] == 'negative':
    return 0

df['sentiment'] = df['review,sentiment'].apply(get_sentiment)

In [81]:
def get_review(x: str) -> int:
  comma_pos = x.rfind(',')
  return x[0:comma_pos]

df['review'] = df['review,sentiment'].apply(get_review)

In [82]:
df = df[['review', 'sentiment']]
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1.0
1,A wonderful little production. <br /><br />The...,1.0
2,I thought this was a wonderful way to spend ti...,1.0
3,Basically there's a family where a little boy ...,0.0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1.0


In [83]:
df.shape

(50000, 2)

In [84]:
df.sentiment.value_counts()

1.0    24978
0.0    24977
Name: sentiment, dtype: int64

In [85]:
df.isna().sum()

review        0
sentiment    45
dtype: int64

In [86]:
# removing nulls
df = df[~df.sentiment.isnull()]

In [87]:
def remove_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  return soup.get_text()

df.review = df.review.apply(remove_html_tags)

def remove_square_brackets(text):
  return re.sub('\[[^]]*\]', '', text)    

df.review = df.review.apply(remove_square_brackets)   

def remove_special_characters(text):
  pattern=r'[^a-zA-z0-9\s]'
  text=re.sub(pattern,'',text)
  return text 
    
df.review = df.review.apply(remove_special_characters)

def stemm(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
  
df.review = df.review.apply(stemm)

tokenizer=ToktokTokenizer()
stopwords_list=stopwords.words('english')
def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  tokens_without_stopwords = [token for token in tokens if token.lower() not in stopwords_list]
  return ' '.join(tokens_without_stopwords)

df.review = df.review.apply(remove_stopwords)

In [88]:
x_train, x_test, y_train, y_test = train_test_split(df.review, df.sentiment, test_size=0.1, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(44959,) (4996,) (44959,) (4996,)


### Calculating SentimentIntensityAnalyzer() Accuracy

In [89]:
sia = SentimentIntensityAnalyzer()

sia_prediction = []
for index, row in x_test.iteritems():
    sentiment = 0.0
    if sia.polarity_scores(row)['compound'] > 0:
        sentiment = 1.0
    sia_prediction.append(sentiment)
accuracy_score(y_test,sia_pred)

0.6563250600480385

### Bag of Words

In [90]:
cv = CountVectorizer(min_df=0, max_df=1, binary=False, ngram_range=(1,3))
x_train_vectorized = cv.fit_transform(x_train)
x_test_vectorized = cv.transform(x_test)

tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
x_train_tv=tv.fit_transform(x_train)
x_test_tv=tv.transform(x_test)

### Modeling

In [92]:
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
lr_bow=lr.fit(x_train_vectorized,y_train)
lr_bow_predict=lr.predict(x_test_vectorized)
lr_bow_score=accuracy_score(y_test,lr_bow_predict)
print("Logistic Regression Score unsing CountVectorizer :",lr_bow_score)

Logistic Regression Score unsing CountVectorizer : 0.5882706164931946


In [93]:
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
lr_bow=lr.fit(x_train_tv,y_train)
lr_bow_predict=lr.predict(x_test_tv)
lr_bow_score=accuracy_score(y_test,lr_bow_predict)
print("Logistic Regression Score using TfidfVectorizer:",lr_bow_score)

Logistic Regression Score using TfidfVectorizer: 0.7327862289831866


In [97]:
dt=DecisionTreeClassifier(criterion='entropy', min_samples_split=4, min_samples_leaf=2, class_weight='balanced', max_depth=8) 
dt_bow=dt.fit(x_train_vectorized,y_train)
dt_bow_predict=dt.predict(x_test_vectorized)
dt_bow_score=accuracy_score(y_test,dt_bow_predict)
print("Decision Tree Score using CountVectorizer :",dt_bow_score)

Decision Tree Score using CountVectorizer : 0.5076060848678943


In [98]:
dt=DecisionTreeClassifier(criterion='entropy', min_samples_split=4, min_samples_leaf=2, class_weight='balanced', max_depth=8) 
dt_bow=dt.fit(x_train_tv,y_train)
dt_bow_predict=dt.predict(x_test_tv)
dt_bow_score=accuracy_score(y_test,dt_bow_predict)
print("Decision Tree Score using TfidfVectorizer :",dt_bow_score)

Decision Tree Score using TfidfVectorizer : 0.5076060848678943


In [101]:
rf=RandomForestClassifier(n_estimators=50, n_jobs=-1)
rf_bow=rf.fit(x_train_vectorized,y_train)
rf_bow_predict=dt.predict(x_test_vectorized)
rf_bow_score=accuracy_score(y_test,rf_bow_predict)
print("Random Forest Score using CountVectorizer :",rf_bow_score)

Random Forest Score using CountVectorizer : 0.5076060848678943
